In [5]:
import numpy as np
import mne
import matplotlib.pyplot as plt
import os

In [6]:
#Takes period segment, voltage threshold, min spike duration in seconds, max spike duration in seconds, and the sampling frequency, returns the amount of spikes in the segment udner those conditions.
def spikeamount(segment, spike_thresh, spike_min, spike_max, samp_freq):
    sample_count = 0
    spikes = 0
    spike_min = spike_min*samp_freq
    spike_max = spike_max*samp_freq
    for sample in segment:
        if sample > spike_thresh:
            sample_count += 1
        elif sample_count > spike_min and sample_count < spike_max:
            spikes += 1
            sample_count = 0
        else:
            sample_count = 0
    return spikes

#Takes a period as list of samples, desired segment length in seconds and the sampling frequency, returns a list of segments of the period {{},{},...{}}
def splitperiod(period, segment_length, sampling_freq):
    segments = []
    current_segment = []
    sample_seglen = segment_length * sampling_freq
    for i, sample in enumerate(period):
        current_segment.append(sample)
        if i%sample_seglen == 0 and i!=0:
            segments.append(current_segment)
            current_segment = []
    return segments
#takes list of spike amounts, returns list of smoothed spike amounts (smoothed spike amount is the average of M neighbour spike amount)
def smoothspikes(spikes, M):
    smoothspikes = []
    window_min = -(M//2)
    window_max = M//2
    for spike in spikes:
        curravg = 0
        if window_min<0:
            for i in range(0,M):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        elif window_max>=len(spikes):
            for i in range (0,M):
                curravg+=spikes[len(spikes)-1-i]
            smoothspikes.append(curravg/M)
        else:
            for i in range(window_min, window_max+1):
                curravg+=spikes[i]
            smoothspikes.append(curravg/M)
        window_min +=1
        window_max +=1
    return smoothspikes


In [11]:
testData = [[1, 2, 3, 23, 24, 23, 23, 24, 3, 4, 24, 32, 42, 3, 12, 32, 32, 3, 23,  32, 32, 3, 23, 23, 13 ,3, 23, 2, 32, 3, 23, 23, 23, 2, 8],[ 12, 3, 2,1, 2, 3, 23, 24, 23, 23, 24, 3, 4, 24, 32, 42, 3, 12, 32, 32, 3, 23,  32, 32, 3, 23, 23, 13 ,3, 23, 2, 32, 3, 23, 23, 23, 2, 8, 12, 3, 2, 2, 3, 23, 24, 23, 23, 24, 3, 4, 24, 32, 42, 3, 12, 32, 32, 3, 23,  32, 32, 3, 23, 23, 13 ,3, 23, 2, 32, 3, 23, 23, 23, 2, 8, 12, 3, 21, 2, 3, 23, 24, 23, 23, 24, 3, 4, 24, 32, 42, 3, 12, 32, 32, 3, 23,  32, 32, 3, 23, 23, 13 ,3, 23, 2, 32, 3, 23, 23, 23, 2, 8, 12, 3, 2]]

testData = [[7 , 8, 1 ,9, 4, 4,4,6,3,4,4,7,7,10,10,4, 2 ,10, 3, 6, 10,1,3,5,6,9,3,10,5,8,6,4,5,8,10,9,2,2,9,7
,1,6,9,4, 3,3,3,4,6,3, 7, 9,2, 8, 7 , 5, 10 , 1, 6, 3, 1 , 6, 9, 6
 ,    9  ,   3  ,   7  ,   2  ,   3  ,   1  ,   8   , 10
 ,    8  ,   8  ,   6  ,   3  ,   3  ,   3  ,  10   ,  3
 ,    6  ,   3  ,   5  ,   2  ,   4  ,   8  ,   9   , 10
 ,    6  ,   9  ,   8  ,   1  ,   2  ,   3 ,    3   ,  2
 ,    3  ,   3  ,  10  ,   4  ,   3  ,   5  ,   4   ,  8
 ,    4  ,   2  ,   7  ,   4  ,   4  ,   1  ,   9   ,  5
 ,    1  ,  10  ,   3  ,   7  ,   7  ,   3  ,  10   ,  7
  ,   6  ,   9  ,   6  ,   1  ,   5  ,   9  ,   8   ,  6
 ,    1  ,   8  ,   5  ,   7  ,   8  ,   9  ,   1   ,  8
 ,    6  ,   1  ,   3  ,   2  ,   6  ,   6  ,   1   , 10
 ,    3  ,   4  ,   4  ,   1  ,   6  ,   3,     7 , 8, 1 ,9, 4, 4,4,6,3,4,4,7,7,10,10,4, 2 ,10, 3, 6, 10,1,3,5,6,9,3,10,5,8,6,4,5,8,10,9,2,2,9,7
,1,6,9,4, 3,3,3,4,6,3, 7, 9,2, 8, 7 , 5, 10 , 1, 6, 3, 1 , 6, 9, 6
 ,    9  ,   3  ,   7  ,   2  ,   3  ,   1  ,   8   , 10
 ,    8  ,   8  ,   6  ,   3  ,   3  ,   3  ,  10   ,  3
 ,    6  ,   3  ,   5  ,   2  ,   4  ,   8  ,   9   , 10
 ,    6  ,   9  ,   8  ,   1  ,   2  ,   3 ,    3   ,  2
 ,    3  ,   3  ,  10  ,   4  ,   3  ,   5  ,   4   ,  8
 ,    4  ,   2  ,   7  ,   4  ,   4  ,   1  ,   9   ,  5
 ,    1  ,  10  ,   3  ,   7  ,   7  ,   3  ,  10   ,  7
  ,   6  ,   9  ,   6  ,   1  ,   5  ,   9  ,   8   ,  6
 ,    1  ,   8  ,   5  ,   7  ,   8  ,   9  ,   1   ,  8
 ,    6  ,   1  ,   3  ,   2  ,   6  ,   6  ,   1   , 10
 ,    3  ,   4  ,   4  ,   1  ,   6  ,   3],[     7 , 8, 1 ,9, 4, 4,4,6,3,4,4,7,7,10,10,4, 2 ,10, 3, 6, 10,1,3,5,6,9,3,10,5,8,6,4,5,8,10,9,2,2,9,7
,1,6,9,4, 3,3,3,4,6,3, 7, 9,2, 8, 7 , 5, 10 , 1, 6, 3, 1 , 6, 9, 6
 ,    9  ,   3  ,   7  ,   2  ,   3  ,   1  ,   8   , 10
 ,    8  ,   8  ,   6  ,   3  ,   3  ,   3  ,  10   ,  3
 ,    6  ,   3  ,   5  ,   2  ,   4  ,   8  ,   9   , 10
 ,    6  ,   9  ,   8  ,   1  ,   2  ,   3 ,    3   ,  2
 ,    3  ,   3  ,  10  ,   4  ,   3  ,   5  ,   4   ,  8
 ,    4  ,   2  ,   7  ,   4  ,   4  ,   1  ,   9   ,  5
 ,    1  ,  10  ,   3  ,   7  ,   7  ,   3  ,  10   ,  7
  ,   6  ,   9  ,   6  ,   1  ,   5  ,   9  ,   8   ,  6
 ,    1  ,   8  ,   5  ,   7  ,   8  ,   9  ,   1   ,  8
 ,    6  ,   1  ,   3  ,   2  ,   6  ,   6  ,   1   , 10
 ,    3  ,   4  ,   4  ,   1  ,   6  ,   3 ,    7 , 8, 1 ,9, 4, 4,4,6,3,4,4,7,7,10,10,4, 2 ,10, 3, 6, 10,1,3,5,6,9,3,10,5,8,6,4,5,8,10,9,2,2,9,7
,1,6,9,4, 3,3,3,4,6,3, 7, 9,2, 8, 7 , 5, 10 , 1, 6, 3, 1 , 6, 9, 6
 ,    9  ,   3  ,   7  ,   2  ,   3  ,   1  ,   8   , 10
 ,    8  ,   8  ,   6  ,   3  ,   3  ,   3  ,  10   ,  3
 ,    6  ,   3  ,   5  ,   2  ,   4  ,   8  ,   9   , 10
 ,    6  ,   9  ,   8  ,   1  ,   2  ,   3 ,    3   ,  2
 ,    3  ,   3  ,  10  ,   4  ,   3  ,   5  ,   4   ,  8
 ,    4  ,   2  ,   7  ,   4  ,   4  ,   1  ,   9   ,  5
 ,    1  ,  10  ,   3  ,   7  ,   7  ,   3  ,  10   ,  7
  ,   6  ,   9  ,   6  ,   1  ,   5  ,   9  ,   8   ,  6
 ,    1  ,   8  ,   5  ,   7  ,   8  ,   9  ,   1   ,  8
 ,    6  ,   1  ,   3  ,   2  ,   6  ,   6  ,   1   , 10
 ,    3  ,   4  ,   4  ,   1  ,   6  ,   3],[     7 , 8, 1 ,9, 4, 4,4,6,3,4,4,7,7,10,10,4, 2 ,10, 3, 6, 10,1,3,5,6,9,3,10,5,8,6,4,5,8,10,9,2,2,9,7
,1,6,9,4, 3,3,3,4,6,3, 7, 9,2, 8, 7 , 5, 10 , 1, 6, 3, 1 , 6, 9, 6
 ,    9  ,   3  ,   7  ,   2  ,   3  ,   1  ,   8   , 10
 ,    8  ,   8  ,   6  ,   3  ,   3  ,   3  ,  10   ,  3
 ,    6  ,   3  ,   5  ,   2  ,   4  ,   8  ,   9   , 10
 ,    6  ,   9  ,   8  ,   1  ,   2  ,   3 ,    3   ,  2
 ,    3  ,   3  ,  10  ,   4  ,   3  ,   5  ,   4   ,  8
 ,    4  ,   2  ,   7  ,   4  ,   4  ,   1  ,   9   ,  5
 ,    1  ,  10  ,   3  ,   7  ,   7  ,   3  ,  10   ,  7
  ,   6  ,   9  ,   6  ,   1  ,   5  ,   9  ,   8   ,  6
 ,    1  ,   8  ,   5  ,   7  ,   8  ,   9  ,   1   ,  8
 ,    6  ,   1  ,   3  ,   2  ,   6  ,   6  ,   1   , 10
 ,    3  ,   4  ,   4  ,   1  ,   6  ,   3 ,     7 , 8, 1 ,9, 4, 4,4,6,3,4,4,7,7,10,10,4, 2 ,10, 3, 6, 10,1,3,5,6,9,3,10,5,8,6,4,5,8,10,9,2,2,9,7
,1,6,9,4, 3,3,3,4,6,3, 7, 9,2, 8, 7 , 5, 10 , 1, 6, 3, 1 , 6, 9, 6
 ,    9  ,   3  ,   7  ,   2  ,   3  ,   1  ,   8   , 10
 ,    8  ,   8  ,   6  ,   3  ,   3  ,   3  ,  10   ,  3
 ,    6  ,   3  ,   5  ,   2  ,   4  ,   8  ,   9   , 10
 ,    6  ,   9  ,   8  ,   1  ,   2  ,   3 ,    3   ,  2
 ,    3  ,   3  ,  10  ,   4  ,   3  ,   5  ,   4   ,  8
 ,    4  ,   2  ,   7  ,   4  ,   4  ,   1  ,   9   ,  5
 ,    1  ,  10  ,   3  ,   7  ,   7  ,   3  ,  10   ,  7
  ,   6  ,   9  ,   6  ,   1  ,   5  ,   9  ,   8   ,  6
 ,    1  ,   8  ,   5  ,   7  ,   8  ,   9  ,   1   ,  8
 ,    6  ,   1  ,   3  ,   2  ,   6  ,   6  ,   1   , 10
 ,    3  ,   4  ,   4  ,   1  ,   6  ,   3]]
#plt.plot(range(0,len(testData[1])),testData[1])
#plt.show()

raw_data = mne.io.read_raw_edf('Data\chb01_02.edf').get_data()

windows = []
for channel in raw_data:
    windows.append(splitperiod(channel,2,256))

for channel in windows:
    segments = [] 
    i = 0
    kanal = []      
    for segment in channel:
        if i<(60/2):
            segments.append(segment)
            i +=1  
        elif i>=(60/2):
            #print(segments) # representattion av 1 min fönstret
            kanal.append(segments)
            #här kan jag köra analys här
            i =0
            segments = []
    # vi har nu summerat kanalen i en kanal som består av kannal [window 1 min [segments 5 sec [1 2 3],....],..........]        
    #print("kanal = "+str(kanal)) 
    #här är analysen då på denna data som är av samma dimension som den förra som var baserad på en kanal updelad i fönster
    spikes = []
    for i, channel in enumerate(kanal):
        spikes.append([])
        for segment in channel:
            spikes[i].append(spikeamount(segment,0.0001,0.04,0.07,256)) # hold up är det inte 20 ms ?
    smoothedspikes = []
    for channel in spikes:
        smoothedspikes.append(smoothspikes(channel,7))

    channel_spike_amounts = []
    
    for channel in smoothedspikes: # skapar en lista för varje kanal, med smoothspike värdena som händer i hela den listan
        spikeamounts = []
        for val in channel:
            if val not in spikeamounts:
                spikeamounts.append(val)
        channel_spike_amounts.append(spikeamounts)

    # sorterar denna lista så vi har det i storleks ordning    
    sorted_channel_spike_amounts=[]
    for channel in channel_spike_amounts:
        channel.sort()
        sorted_channel_spike_amounts.append(channel)
    
    channel_window_amounts = []
   
    channel_window_amounts = []
    # tar sorterade listan och skappar en lista på i hur många windows deta värde sker i 
    for i,channel in enumerate(smoothedspikes):
        channel_window_amounts.append([0] * len(channel_spike_amounts[i]))
        print(channel_window_amounts) # printat hur långt den kommer
        for val in channel:
            for window in sorted_channel_spike_amounts:
                if val in window:
                    channel_window_amounts[i][window.index(val)] += 1
                    
    
       
    ## så här ovan är det mest framgångsrika försöket som basicly är copy pastad från hur den gjordes när jag ej delat in i 1 min windows
    
    
    ###################
    # under är en annan metod jag putsat på
    # Datan ser liknande ut som när jag gjorde på föra sättet men nu verkar det som att vid ett vist index funkar det ej längre
    # har även en annan metod att skappa en lista lika stor som den för sorted men med nollor men lyckas ej få till countern
    
    # for channel in sorted_channel_spike_amounts:
    #    channel_window_amounts.append([0]*len(channel))  
    
    #försökte köra en counter här som sen går igenom och räknar
    
    
    #for i,channel in enumerate(sorted_channel_spike_amounts):
    #    print(i)
    #    print(channel)
    #    for val in channel:
    #        for smoothchannel in smoothedspikes:
    #            if val in smoothchannel:
    #                print("i see you")     
    
    # men får som sagt ej till countern
    
        

Extracting EDF parameters from d:\Epilepsi spike detection\Data\chb01_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\johan\AppData\Local\Temp\ipykernel_9680\3666729363.py:79: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data = mne.io.read_raw_edf('Data\chb01_02.edf').get_data()


[[0, 0, 0]]
[[584, 578, 144], [0, 0]]
[[584, 578, 144], [1318, 280], [0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [0, 0, 0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [1086, 138, 148, 30], [0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [1086, 138, 148, 30], [1318, 280], [0, 0, 0, 0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [1086, 138, 148, 30], [1318, 280], [637, 412, 104, 47, 16], [0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [1086, 138, 148, 30], [1318, 280], [637, 412, 104, 47, 16], [926, 560], [0, 0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [1086, 138, 148, 30], [1318, 280], [637, 412, 104, 47, 16], [926, 560], [917, 227, 216], [0, 0, 0, 0, 0]]
[[584, 578, 144], [1318, 280], [1318, 280], [1318, 280], [1086, 138, 148, 30], [1318, 280], [637, 412, 104, 47, 16], [926, 560], [917, 227, 216], [71, 569, 90, 96, 48], [0, 0, 

IndexError: list index out of range